In [1]:
import numpy as np
from pathlib import Path

from pymor.basic import *
from pymor.core.pickle import load

from RBInvParam.problems.problems import build_InstationaryModelIP

set_log_levels({
    'pymor' : 'WARN'
})

set_defaults({})


In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "cm",
    "font.size": 10,
    'text.latex.preamble': r'\usepackage{amsfonts} \usepackage{accents} \usepackage{mathrsfs}',
    'figure.dpi': 400
})

In [3]:
WORK_DIR = Path('/run/media/benedikt/38E84473425D444F/Dokumente/Studium/Mathematik/PhD/parabolische_inverse_probleme')
#WORK_DIR = Path('/home/benedikt/Dokumente/parabolische_inverse_probleme')
#data_dir_path = WORK_DIR / Path('experiments/corrected_norm_non_timedep')
data_dir_path = WORK_DIR / Path('experiments/corrected_norm_full_timedep')

FOM_data_path = data_dir_path / 'diffusion_FOM_300'
TR_data_path = data_dir_path / 'diffusion_TR_300'

setup_path = FOM_data_path / 'setup.pkl'
optimizer_parameter_path = FOM_data_path / 'optimizer_parameter.pkl'


with open(setup_path, 'rb') as file:
    setup = load(file)

with open(optimizer_parameter_path, 'rb') as file:
    optimizer_parameter = load(file)

if not 'FOM' in locals():
    FOM, grid_data, assembled_products = build_InstationaryModelIP(setup=setup)

    

data = {}

if FOM_data_path:
    #with open(FOM_data_path / 'FOM_IRGNM_final.pkl', 'rb') as file:
    with open(FOM_data_path / 'FOM_IRGNM_5.pkl', 'rb') as file:
        FOM_data = load(file)
    data['FOM'] = FOM_data

if TR_data_path:
    with open(TR_data_path / 'TR_IRGNM_final.pkl', 'rb') as file:
        TR_data = load(file)
    data['TR'] = TR_data
    


[2025-03-14 14:21:24,866][__init__] - Setting up InstationaryModelIP
[2025-03-14 14:22:20,118][discretize_instationary_IP] - noise percentage is 7.0703e-08
[2025-03-14 14:22:20,125][discretize_instationary_IP] - noise_level is 1.0000e-05
[2025-03-14 14:22:20,282][__init__] - Setting up InstationaryModelIP


In [4]:
data['TR']['FOM_num_calls']

{'solve_state': 12,
 'solve_adjoint': 12,
 'solve_linearized_state': 0,
 'solve_linearized_adjoint': 0,
 'objective': 12,
 'gradient': 12,
 'linearized_objective': 0,
 'linearized_gradient': 0}

In [5]:
import pandas as pd

columns = ['Algorithm', 'time [s]', 'speed up', 'FOM solves', r'$n_Q$', r'$n_V$', 'o. iter']

rows = []
algo = 'FOM'
FOM_solves = data[algo]['FOM_num_calls']['solve_state'] + data[algo]['FOM_num_calls']['solve_adjoint'] + \
             data[algo]['FOM_num_calls']['solve_linearized_state'] + data[algo]['FOM_num_calls']['solve_linearized_adjoint']
row = [algo,
       int(data[algo]['total_runtime'][-1]),
       '--',
       FOM_solves,
       '--',
       '--',
       len(data[algo]['J'])-1]
rows.append(row)

algo = 'TR'
FOM_solves = data[algo]['FOM_num_calls']['solve_state'] + data[algo]['FOM_num_calls']['solve_adjoint'] + \
             data[algo]['FOM_num_calls']['solve_linearized_state'] + data[algo]['FOM_num_calls']['solve_linearized_adjoint']

row = [algo,
       int(data[algo]['total_runtime'][-1]),
       '--',
       FOM_solves,
       data[algo]['dim_Q_r'][-1],
       data[algo]['dim_V_r'][-1],
       len(data[algo]['J'])-1]
rows.append(row)


df = pd.DataFrame.from_records(rows, columns=columns)
df['speed up'][1] = df['time [s]'][0] / df['time [s]'][1]

/tmp/ipykernel_26624/2752217702.py:33: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['speed up'][1] = df['time [s]'][0] / df['time [s]'][1]
/tmp/ipykernel_26624/2752217702.py:33: SettingWithCopyWarning: 
A value is trying to be set on a c

In [6]:
print(df.to_latex(index=False, float_format="{:.2f}".format, column_format='l|cccccc'))

\begin{tabular}{l|cccccc}
\toprule
Algorithm & time [s] & speed up & FOM solves & $n_Q$ & $n_V$ & o. iter \\
\midrule
FOM & 30613 & -- & 30168 & -- & -- & 5 \\
TR & 93250 & 0.33 & 24 & 404 & 664 & 8 \\
\bottomrule
\end{tabular}



In [9]:
345600 / 93250

3.706166219839142

In [7]:
4*24*60*60

345600